In [5]:
import pandas as pd
df = pd.read_csv(r'D:\Practice_Project\NLP\Risk_Analysis\data\aggressive_train_data.csv',encoding='latin1')
df.head()

,TweetText,label
0,criminally underrated movie basically western ...,1
1,dark skin people dont need paint face black sp...,1
2,lol mm ramen thats college special,0
3,yes pure evil love anyway hey know aussie rule,0
4,like burnin way beyond midnight oil wonderful ...,0


In [6]:
df.shape

(128912, 2)

In [7]:
X=list(df['TweetText'])

In [8]:
y=list(df['label'])

In [9]:
X[1]

'dark skin people dont need paint face black spread message laugh colorism fuckin faggot'

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [11]:
X_train

['new song amazing cant wait buy album counting day',
 'new post newi community exonerates senator ifeanyi ubah slam traducer sonacobary',
 'dont forget frosting cow pie left would anyone ever vote man oversaw theft million cm people florida care medicare',
 'luke nkjv pharisee stood prayed thus god thank like menextortioners unjust adulterer even tax collector',
 'college rly said self plagiarism still plagiarism copy idea previous presentation another one like',
 'literally used word faggot first tweet highly offensive homophobic slur praise art literally insulted people doe matter intention wa wa apologize',
 'oh without time quot like anytime doin favorite hobby thing change',
 'pathetic dnt guy opposition moral right accuse another political party criminality executed impunity crashing fast',
 'imagine walkin work significant friend light skin white person yell stupid nigger car think nun wrong even tho biracial friend new wasup stuck man couldnt',
 'let welcome brand new need fol

In [12]:
!pip install transformers

In [13]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [14]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [15]:
y_train

[0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,


In [16]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [17]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.019192969799041747}

In [ ]:
trainer.predict(test_dataset)

array([0, 1, 0, ..., 0, 1, 0], dtype=int32)

In [ ]:
trainer.predict(test_dataset)[1].shape

(1115,)

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[955,   0],
       [  0, 160]])

In [ ]:
trainer.save_model('risk_bert_model')